<a href="https://colab.research.google.com/github/daonly/2023Autumn/blob/main/2022021241_%EC%9D%B4%EB%8B%A4%EC%98%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive # 코랩과 구글드라이드를 연동해줍니다.
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
import os
os.chdir("/gdrive/My Drive")

## Preliminary

In [4]:
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
np.random.seed(1211)

import time

from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

from dataset.mnist import load_mnist
from dataset.fashion_mnist import load_fashion_mnist
from dataset.cifar_10 import load_cifar_10

In [5]:
# =============================================================================
# 곱셈 계층
# =============================================================================
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y  # x와 y를 바꾼다.
        dy = dout * self.x

        return dx, dy

# =============================================================================
# 덧셈 계층
# =============================================================================
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

# =============================================================================
# ReLU 계층
# =============================================================================
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

# =============================================================================
# Sigmoid 계층
# =============================================================================
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x) # 순전파의 출력을 out에 보관한 후 역전파 계산 때 그 값을 사용
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

# =============================================================================
# Affine 계층
# =============================================================================
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b

        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

# =============================================================================
# Softmax-with-Loss 계층
# =============================================================================
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size

        return dx

# =============================================================================
# 오차역전파를 적용한 신경망
# =============================================================================
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

### 오차역전차법을 사용한 학습 예시

In [ ]:
# # 데이터 읽기
# (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# iters_num = 10000
# train_size = x_train.shape[0]
# batch_size = 100
# learning_rate = 0.1

# train_loss_list = []
# train_acc_list = []
# test_acc_list = []

# iter_per_epoch = max(train_size / batch_size, 1)

# for i in range(iters_num):
#     batch_mask = np.random.choice(train_size, batch_size)
#     x_batch = x_train[batch_mask]
#     t_batch = t_train[batch_mask]

#     # 기울기 계산
#     #grad = network.numerical_gradient(x_batch, t_batch)
#     grad = network.gradient(x_batch, t_batch)

#     # 갱신
#     for key in ('W1', 'b1', 'W2', 'b2'):
#         network.params[key] -= learning_rate * grad[key]

#     loss = network.loss(x_batch, t_batch)
#     train_loss_list.append(loss)

#     if i % iter_per_epoch == 0:
#         train_acc = network.accuracy(x_train, t_train)
#         test_acc = network.accuracy(x_test, t_test)
#         train_acc_list.append(train_acc)
#         test_acc_list.append(test_acc)
#         print(train_acc, test_acc)

# Q1. 제공된 데이터 세트로 변경하여 다음 코드를 실행하고, 결과를 저장하시오.

In [ ]:
(x_train, t_train), (x_test, t_test) = load_fashion_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.gradient(x_batch, t_batch)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

x_train has changed completely
x_test has changed completely
t_train has changed completely
t_test has changed completely
0.1837 0.1869
0.81174 0.8094
0.82772 0.823
0.84482 0.8392
0.84274 0.8394
0.85772 0.8496
0.86342 0.8564
0.868 0.8585
0.87596 0.8666
0.87578 0.8662
0.87534 0.8654
0.8736 0.8629
0.88372 0.871
0.88658 0.8734
0.88778 0.8711
0.88612 0.8733
0.89338 0.876
0.89288 0.8768
0.892 0.8736
0.89516 0.8759


In [ ]:
(x_train, t_train), (x_test, t_test) = load_cifar_10(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=1024, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.gradient(x_batch, t_batch)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

x_train has changed completely
x_test has changed completely
t_train has changed completely
t_test has changed completely
0.0985 0.0987
0.31795 0.3099
0.369475 0.3578
0.374125 0.3593
0.400275 0.3892
0.433625 0.4184
0.388375 0.3738
0.419775 0.4079
0.449375 0.4321
0.4239 0.4051
0.466425 0.4416
0.46185 0.4379
0.47985 0.459
0.4776 0.4556
0.4786 0.4496
0.46995 0.4498
0.48915 0.4668
0.485525 0.4642
0.481975 0.4518
0.46505 0.4413
0.4957 0.4677
0.5072 0.4762
0.47445 0.4447
0.4799 0.4484
0.509075 0.4747


# Q2. 신경망 학습에서 학습하는데 한 번에 쓸 수 있는 데이터의 크기를 결정하는 'batch size'와 weight를 업데이트하는데 사용되는 'learning rate'는 학습 결과에 영향을 주는 중요한 하이퍼 파라미터이다. 다음 [batch_size, learning_rate] 가운데 1개를 선택해 수정하고, 수정 전 코드의 실행 결과와 비교하여 발생되는 차이점을 작성하시오.

## 변경 전 코드

In [ ]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.gradient(x_batch, t_batch)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.09375 0.0941
0.9042333333333333 0.9089
0.9238833333333333 0.9264
0.9362333333333334 0.9364
0.9447833333333333 0.9436
0.9501333333333334 0.9473
0.9552333333333334 0.9526
0.9592833333333334 0.956
0.9641166666666666 0.9612
0.9671166666666666 0.9625
0.9693 0.9653
0.97195 0.9676
0.9728833333333333 0.9665
0.9761 0.9688
0.97695 0.9694
0.9786333333333334 0.9714
0.979 0.9718


## 변경 후 코드

In [ ]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100                                             # <== 이 부분을 수정하면 됨
learning_rate = 0.01                                             # <== 이 부분을 수정하면 됨

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.gradient(x_batch, t_batch)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.032466666666666665 0.0314
0.5902166666666666 0.5949
0.7792333333333333 0.7847
0.8386333333333333 0.845
0.86575 0.872
0.8796833333333334 0.8846
0.8889 0.8927
0.8949 0.8983
0.8989833333333334 0.9024
0.90295 0.9057
0.90585 0.9099
0.9082166666666667 0.9117
0.9105 0.9142
0.91245 0.9169
0.9138666666666667 0.9156
0.91555 0.9193
0.9175333333333333 0.9202


In [ ]:
print("learning rate을 10분의 1로 줄여서 비교해보았다. 확실히 같은 횟수의 iteration을 돌았을 때 Acc 값의 차이로 보아, 학습이 더디게 진행되는 것을 확인할 수 있었다.")       # <= 이 부분에 답안을 작성

learning rate을 10분의 1로 줄여서 비교해보았다. 확실히 같은 횟수의 iteration을 돌았을 때 Acc 값의 차이로 보아, 학습이 더디게 진행되는 것을 확인할 수 있었다.


# Q3. 기울기 계산 방법에 대해 '수치 미분 방식'과 '오차역전파 방식'을 개별 실험하고, 계산 소요 시간 차이에 대한 원인과 이유를 작성하시오.
	# Tips: 이 문제는 코드의 실행 여부를 확인하며, 본인의 생각을 자유롭게 작성하시면 됩니다.

In [ ]:
# 코드 실행 전 시간 기록
start_time = time.time()

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식                                             # <== 이 부분을 수정하면 됨
#   grad = __________________________________________ # 오차역전파법 방식                                             # <== 이 부분을 수정하면 됨

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

# if i % iter_per_epoch == 0:
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    train_acc_list.append(train_acc)
    test_acc_list.append(test_acc)
    print(train_acc, test_acc, "--" "Itertation_number =", i)

# 코드 실행 후 시간 기록
end_time = time.time()

# 실행 시간 계산
execution_time = end_time - start_time
print(f"수치 미분 방식 코드 실행 시간: {execution_time} 초")

0.08618333333333333 0.0868 --Itertation_number = 0
0.11161666666666667 0.1137 --Itertation_number = 1
0.11748333333333333 0.1215 --Itertation_number = 2
0.1038 0.1028 --Itertation_number = 3
0.10253333333333334 0.1016 --Itertation_number = 4
0.1258 0.1236 --Itertation_number = 5
0.1195 0.1188 --Itertation_number = 6
0.10461666666666666 0.1039 --Itertation_number = 7
0.10608333333333334 0.1056 --Itertation_number = 8
0.13701666666666668 0.1332 --Itertation_number = 9
수치 미분 방식 코드 실행 시간: 844.1539824008942 초


In [15]:
# 코드 실행 전 시간 기록
start_time = time.time()

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
#   grad = __________________________________________ # 수치 미분 방식                                             # <== 이 부분을 수정하면 됨
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식                                             # <== 이 부분을 수정하면 됨

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc, "--" "Itertation_number =", i)

# 코드 실행 후 시간 기록
end_time = time.time()

# 실행 시간 계산
execution_time = end_time - start_time
print(f"오차역전파 코드 실행 시간: {execution_time} 초")

0.1752 0.18 --Itertation_number = 0
0.8999833333333334 0.9044 --Itertation_number = 600
0.9197 0.9221 --Itertation_number = 1200
0.9321333333333334 0.9354 --Itertation_number = 1800
0.9412166666666667 0.9409 --Itertation_number = 2400
0.949 0.9467 --Itertation_number = 3000
0.9543 0.952 --Itertation_number = 3600
0.95765 0.9548 --Itertation_number = 4200
0.9634333333333334 0.9599 --Itertation_number = 4800
0.9656666666666667 0.9593 --Itertation_number = 5400
0.9687 0.9632 --Itertation_number = 6000
0.9706666666666667 0.9652 --Itertation_number = 6600
0.97305 0.9667 --Itertation_number = 7200
0.9748166666666667 0.9674 --Itertation_number = 7800
0.97695 0.9688 --Itertation_number = 8400
0.9772666666666666 0.9691 --Itertation_number = 9000
0.9797666666666667 0.9701 --Itertation_number = 9600
오차역전파 코드 실행 시간: 27.092668294906616 초


In [1]:
print("확실히 수치 미분 방식에 비해 오차역전파 방식이 미분을 효울적으로 할 수 있기 때문에 훨씬 빠르게 결과를 내는 것을 확인할 수 있었다.")       # <= 이 부분에 답안을 작성

확실히 수치 미분 방식에 비해 오차역전파 방식이 미분을 효울적으로 할 수 있기 때문에 훨씬 빠르게 결과를 내는 것을 확인할 수 있었다.


# Q4. 다음 class에서 ReLU 활성화 함수를 sigmoid로 변경하여 코드를 실행하고, 각 활성화 함수에 대한 특징을 작성하시오.

In [6]:
class TwoLayerNet_Sigmoid:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Sigmoid1'] = Sigmoid()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [7]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet_Sigmoid(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.gradient(x_batch, t_batch)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.09751666666666667 0.0974
0.773 0.7832
0.87595 0.8806
0.89725 0.9014
0.9068 0.911
0.9127 0.9155
0.9189166666666667 0.9222
0.9232166666666667 0.9258
0.927 0.929
0.93115 0.9316
0.9346166666666667 0.9346
0.9366833333333333 0.9362
0.9395666666666667 0.94
0.9403833333333333 0.9405
0.9432 0.9433
0.9450166666666666 0.9449
0.9466 0.9444


In [16]:
print("Sigmoid의 vanishing gradient problem을 해결하기 위해 고안된 것이 ReLU이다. 다만 본인의 경우 이 코드에서는 ReLU보다 Sigmoid가 비슷한 소요 시간에 비해 더 정확한 퍼포먼스를 보여주는 것을 확인할 수 있었다. 따라서 본 데이터 분석에서는 vanishing gradient problem이 발생하지 않았던 것으로 사료된다.")       # <= 이 부분에 답안을 작성

Sigmoid의 vanishing gradient problem을 해결하기 위해 고안된 것이 ReLU이다. 다만 본인의 경우 이 코드에서는 ReLU보다 Sigmoid가 비슷한 소요 시간에 비해 더 정확한 퍼포먼스를 보여주는 것을 확인할 수 있었다. 따라서 본 데이터 분석에서는 vanishing gradient problem이 발생하지 않았던 것으로 사료된다.


# Q5. 다음 class에서 Affine layer를 한 층 더 추가하여 코드를 실행하고, 이에 따른 결과를 기존 실행 결과와 비교하시오.
    (두 번째 활성화 함수도 동일하게 ReLU를 사용할 것)
    (첫 번째 hidden layer의 size는 100, 두 번째 hidden layer의 size는 50으로 설정할 것)
	# Tip: 제공된 code의 주석 부분에 대해 layer를 추가함으로써 Weight 파라미터에 대해 생각하면 좋을 것 같습니다.

In [13]:
class ThreeLayerNet:

    def __init__(self, input_size, hidden_size, hidden_size2, output_size, weight_init_std = 0.01):        # <== 이 부분을 수정 또는 추가하면 됨
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, hidden_size2)
        self.params['b2'] = np.zeros(hidden_size2)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size2, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        grads['W3'] = numerical_gradient(loss_W, self.params['W3'])
        grads['b3'] = numerical_gradient(loss_W, self.params['b3'])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W3'], grads['b3'] = self.layers['Affine3'].dW, self.layers['Affine3'].db

        return grads

In [14]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = ThreeLayerNet(input_size=784, hidden_size=100, hidden_size2=50, output_size=10) # <== 이 부분을 수정 또는 추가하면 됨

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.gradient(x_batch, t_batch)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):                      # <== 이 부분을 수정 또는 추가하면 됨
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.09615 0.0932
0.77895 0.7853
0.9054166666666666 0.902
0.9360833333333334 0.9336
0.9516333333333333 0.947
0.9616333333333333 0.9566
0.9654333333333334 0.9608
0.9723666666666667 0.9656
0.9767333333333333 0.9695
0.9773666666666667 0.9688
0.9812833333333333 0.9707
0.9840833333333333 0.9721
0.9845333333333334 0.9718
0.9851 0.9726
0.9881333333333333 0.9756
0.9881833333333333 0.9725
0.9899333333333333 0.974


In [17]:
print("Layer를 하나 더 늘리니 확실히 조금 더 오랜 시간이 소요되긴 하였으나, 더 높은 정확도를 확인할 수 있었다.")       # <= 이 부분에 답안을 작성

Layer를 하나 더 늘리니 확실히 조금 더 오랜 시간이 소요되긴 하였으나, 더 높은 정확도를 확인할 수 있었다.
